In [23]:
# See https://pypsa.readthedocs.io/en/latest/user-guide/contingency-analysis.html#branch-outage-distribution-factors-bodf
import polars as pl

from benchmarks.utils import mock_snakemake

DF_CUTOFF = 1e-5


if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("compute_bodf")


lines = pl.read_parquet(snakemake.input.lines)
lines = lines.filter(~pl.col("is_leaf")).select("line_id", "from_bus", "to_bus")
ptdf = pl.read_parquet(snakemake.input.ptdf)
lines

line_id,from_bus,to_bus
i64,i64,i64
9181,7168,7170
6839,5769,5770
7110,2059,8570
1577,3139,3180
2379,4286,4290
…,…,…
10753,8739,8740
2936,3822,3823
1535,3404,4312


In [24]:
bptdf = (
    lines.rename({"line_id": "outage"})
    .join(
        ptdf.rename({"injection": "from_bus", "factor": "injection"}),
        on="from_bus",
    )
    .join(
        ptdf.rename({"injection": "to_bus", "factor": "withdrawal"}),
        on=["to_bus", "line"],
    )
    .select(
        "outage",
        "line",
        factor=pl.col("injection") - pl.col("withdrawal"),
    )
)
bptdf

outage,line,factor
i64,i64,f64
1227,3,0.000365
2794,3,-0.00003
2796,3,-0.000024
10821,3,-0.000005
5267,3,2.4301e-7
…,…,…
1224,10821,-2.9233e-8
8558,10821,1.7764e-15
7688,10821,0.000003


In [25]:
diagonal_entries = bptdf.filter(pl.col("outage") == pl.col("line"))
offdiagonal_entries = bptdf.filter(pl.col("outage") != pl.col("line"))
diagonal_entries

outage,line,factor
i64,i64,f64
3,3,-0.167553
6,6,0.072846
8,8,-1.5068e-9
9,9,0.08124
10,10,0.315663
…,…,…
10815,10815,0.000007
10818,10818,-3.0739e-8
10819,10819,0.100823


In [26]:
bridge_lines = diagonal_entries.filter(pl.col("factor") >= (1 - DF_CUTOFF))
if bridge_lines.height > 0:
    raise ValueError(
        f"Grid is not well connected\n{diagonal_entries.filter(pl.col('factor') >= (1 - DF_CUTOFF))}"
    )

In [27]:
bodf = (
    offdiagonal_entries.join(
        diagonal_entries.select("outage", denominator=1 - pl.col("factor")),
        on="outage",
    )
    .with_columns(pl.col("factor") / pl.col("denominator"))
    .drop("denominator")
)
bodf

outage,line,factor
i64,i64,f64
1227,3,0.000365
2794,3,-0.000029
2796,3,-0.000025
10821,3,-0.000005
5267,3,1.6497e-7
…,…,…
1204,10821,1.7282e-8
1224,10821,-2.9101e-8
8558,10821,1.7764e-15


In [28]:
# Filter out near zeros
bodf = bodf.filter(pl.col("factor").abs() > DF_CUTOFF)
bodf

outage,line,factor
i64,i64,f64
1227,3,0.000365
2794,3,-0.000029
2796,3,-0.000025
1673,3,0.000864
1674,3,-0.004317
…,…,…
723,10821,0.000208
35,10821,-0.000109
724,10821,-0.000639


In [12]:
bodf.write_parquet(snakemake.output[0])